# 🧠 Clasificación de Productos - Google Colab
Este notebook carga datos desde Google Drive, entrena una red neuronal convolucional (CNN) y evalúa su desempeño.

In [1]:
# 📂 Montar Google Drive con dataset ya preprocesado
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# 🚀 Clonar repositorio
!git clone https://github.com/solivare/DeepNeuralNetworkUSS.git
%cd DeepNeuralNetworkUSS/Projects/Supermarket/

fatal: destination path 'DeepNeuralNetworkUSS' already exists and is not an empty directory.
/content/DeepNeuralNetworkUSS/Projects/Supermarket


In [3]:
# 📦 Instalar dependencias
!pip install -r requirements.txt

In [4]:
# 📁 Ajustar paths para importar módulos
import os, sys
src_path = os.path.abspath('src')
if src_path not in sys.path:
    sys.path.append(src_path)

## 🧠 Entrenamiento del Modelo

In [ ]:
# 🔧 Cargar config y actualizar ruta a Drive
from src import train, model, utils
import yaml

with open("config.yaml") as f:
    config = yaml.safe_load(f)

# Sobrescribir ruta para correr desde Drive
config["paths"]["prepared_data"] = "/content/drive/MyDrive/DeepNeuralNetworkUSS/Data/Supermarket/items_prepared"

# 🧠 Entrenamiento
train_gen, val_gen = train.get_data_generators(config)
num_classes = len(train_gen.class_indices)

cnn_model = model.build_model(config, num_classes)
history = cnn_model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=config['training']['epochs']
)

utils.plot_training_history(history)

Found 1592 images belonging to 54 classes.
Found 355 images belonging to 54 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/2
24/50 ━━━━━━━━━━━━━━━━━━━━ 7:46 18s/step - accuracy: 0.0291 - loss: 4.3483

## 📊 Evaluación del Modelo

In [ ]:
from src import evaluate as ev
import numpy as np
from sklearn.metrics import classification_report

y_true = val_gen.classes
y_scores = cnn_model.predict(val_gen)
y_pred = np.argmax(y_scores, axis=1)
class_names = list(val_gen.class_indices.keys())

ev.plot_confusion_matrix(y_true, y_pred, class_names)
print(classification_report(y_true, y_pred, target_names=class_names))